### Menoplan Fraud Detection Protocol

In [142]:
import pandas as pd # Pandas is an open source library providing data structures and data analysis tools for Python
import re #regex
from functools import reduce 
import numpy as np
from math import isnan
import matplotlib.pyplot as plt
import urllib.request
import json
from datetime import date
from datetime import datetime
today = '14jan'

In [143]:
beta_file_loc =  "C:/Users/huixin/OneDrive - UC San Diego/menoplan/pilot_phase/merged_analysis/"
pilot_file_loc = "C:/Users/huixin/OneDrive - UC San Diego/menoplan/trial_phase/"
qualtrics_data = pd.read_csv(pilot_file_loc + "qualtrics_data/" + today + "/"+ today + "_p2.csv")
screening = pd.read_csv(pilot_file_loc +"qualtrics_data/" + today + "/"+ today + "_p1.csv")
screening=screening.iloc[2:] #drop first two rows
qualtrics_data=qualtrics_data.iloc[2:] #drop first two rows

#helper functions
def straightLining(condDf,  rowNum, colNames): #check if answers to questions in a row are the same
    elems = list((condDf[colNames].eq(condDf[colNames].iloc[rowNum:rowNum+1, 0], axis=0).values)[0])
    return elems.count(elems[0]) == len(elems) # are all elements in the list same?

def replaceEmptyWithNAN(varname, dfName): #replace empty string with NAN and convert col to numeric
    dfName[varname].replace(" ", np.NaN, regex=True)
    dfName[varname]=pd.to_numeric(dfName[varname])

# req = urllib.request.Request('https://raw.githubusercontent.com/millbj92/US-Zip-Codes-JSON/master/USCities.json')
# with urllib.request.urlopen(req) as response: #master file for matching zipcode to 
#     js = json.load(response) 
    
def findState(zc):
    for dct in js:
        if dct.get("zip_code") == zc:
            return dct.get('state')
        
def containQuoteMarks(string):
    if string.startswith("\"") and string.endswith("\""):
        return True 
def getLsLen(string):
    li = list(string.split(","))
    return len(li)

# #data cleaning
ga_data = pd.read_csv(beta_file_loc + "02nov_merged_pageviews_events_os.csv")
qualtrics_data.columns = qualtrics_data.columns.str.replace('Infromation', 'Information')
qualtrics_data.loc[qualtrics_data['C-Timing_First Click'].isna() , 'Cond'] = 'Exp' #create new col to indicate condition
qualtrics_data.loc[~qualtrics_data['C-Timing_First Click'].isna() , 'Cond'] = 'Ctrl'
qualtrics_data = qualtrics_data.rename(columns= #rename cols
                                       {'RecordedDate': 'EndVisitTs',
                                        'Duration (in seconds)': 'durationMins' }) 

In [144]:
#convert to numeric, replace ' ' with NAN
ls = ['C-Gender', 'E-gender', 'Q_RecaptchaScore','Randomization_2','Randomization_1', 'C-ATT-CHECK1', 
      'C-ATT-CHECK2', 'E-ATT-CHECK1', 'E-ATT-CHECK2',
    #  'C-Age', 'E-Age',  use birthdate var instead
      'durationMins'] #convert to numeric
for x in ls:
    replaceEmptyWithNAN(x, qualtrics_data)
for x in ['Q_RelevantIDDuplicateScore','Q_RelevantIDFraudScore' ]:
    replaceEmptyWithNAN(x, screening)
     
qualtrics_data['durationMins'] = qualtrics_data['durationMins']/60 #convert duration from secs to mins
first_column =qualtrics_data.pop('Cond')# shift column 'Cond' to first position
qualtrics_data.insert(0, 'Cond', first_column)  
qualtrics_data = qualtrics_data.add_suffix('_p2') #add _p2 to each col to indicate this is part 2 of survey

### Inattention
1. Unusually short period of time exploring the study websites (&lt; 10 minutes) `p1p2df['StartDate_p2']- p1p2['EndDate']`  
    - Check the timing of when they start answering this question. and when they first entered the survey: Please think about your experience with the information on the website. How much do you agree or disagree with each statement?
2. Unusually short survey completion times (&lt; 2 minutes)  `['durationMins']`
3. Selection of all items in multiple response questions 
    - find variable names of multiple-response questions
    - check if all answers were selected  `selectAll`
4. Selection of same response to all items in a list of possible items (straight lining) 
    - find variable names of multiple-response questions 
        - Check if same `straightLining
5. Failure to answer attention check correctly.`['Q_RecaptchaScore'] >= 80`

#### Duplicate or unusual responses to open-ended items
1. Several responses follow the same pattern in terms of phrasing or format  
    - check if open ended answers are duplicated
2. Nonsensical or non-matching response to open ended questions 
    - check manually
    
#### Other inconsistent or unusual responses
1. High level of nonresponse to survey questions or refusal, >10% - Done 2. Mismatched survey answers (for example 2 different menopause stages, presence/absence of menopause symptoms)  - Done
3. Age out of range (&lt;40, &gt; 55) - Done
4. Male sex - Done
5. Indicated wrong study source - TBA
6. Look to see if strange links are sites that direct people to make money 
    - check traffic from GA for Menoplan Website
7. Duplicate open-ended responses - Done

#### Inconsistent responses to verifiable items
1. Suspicious email address (for example at least 10 random numbers or letters in a row or strange punctuation or capitalization) `['?-Email_p2'].str.isupper()`

2. Time of submission (for example, 12-4 am)
3. Business vs personal address - Manual Check, Spokeo
4. Address not real - Manual Check, Spokeo
5. Address includes quotations marks - Done
6. Name/suffix suspicious - Manual Check, Spokeo
7. Business vs. personal phone number - Manual Check, Spokeo
8. Entered zipcode and state mismatch - Done
9. IP address not in the U.S. - Manual Check, Spokeo 
10. IP address appears on more than one entry 

In [145]:
conDf = qualtrics_data[list(qualtrics_data.iloc[:,:185].columns)+list(qualtrics_data.iloc[:, -5:])] #subset only columns relevant to control condition (C-)
expDf = qualtrics_data[list(qualtrics_data.iloc[:, :30].columns) +  list(qualtrics_data.iloc[:, 185:].columns)]

In [146]:
conDf =conDf[conDf['Cond_p2'] == 'Ctrl'] #subset only  rows from control condition participants
expDf =expDf[expDf['Cond_p2'] == 'Exp'] #subset only rows from experiment condition participants
expDf

,Cond_p2,StartDate_p2,EndDate_p2,Status_p2,IPAddress_p2,Progress_p2,durationMins_p2,Finished_p2,EndVisitTs_p2,ResponseId_p2,...,E-Job_Status_p2,E-Basics_p2,E-Website_Link_p2,E-Feedback_p2,E-Open Question_p2,QCity_ExportTag_p2,cityName_p2,Email_p2,Q_BallotBoxStuffing_p2,Referer_p2
2,Exp,2022-01-14 00:01:58,2022-01-14 00:35:56,0,76.104.216.47,100,33.950000,1,2022-01-14 00:35:56,R_ReNNNAOqMc3gL6h,...,1,1,1,NaN,I didn't like the icons/graphics,NaN,city =Bellingham,NaN,NaN,android-app://com.google.android.gm/
4,Exp,2022-01-14 03:09:44,2022-01-14 03:39:15,0,105.161.171.91,100,29.516667,1,2022-01-14 03:39:16,R_3NQg6A6Dd7X7toQ,...,1,1,1,Everything is great,You have done a great job,NaN,city =New York,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_74...
5,Exp,2022-01-14 03:42:19,2022-01-14 04:24:43,0,86.106.87.110,100,42.383333,1,2022-01-14 04:24:43,R_3kHtyBUTqw6U7re,...,1,1,1,I am satisfied,Great work!,NaN,city =Miami,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_74...
10,Exp,2022-01-14 07:56:19,2022-01-14 11:15:09,0,76.183.86.133,100,198.833333,1,2022-01-14 11:15:10,R_26aobOw7DVENare,...,3,1,1,What happens if menopause starts early in life...,None,NaN,city =Dallas,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_74...
11,Exp,2022-01-14 08:45:12,2022-01-14 11:51:16,0,172.58.107.19,100,186.066667,1,2022-01-14 11:51:17,R_3GkW8Tq5b0KTlLB,...,5,2,1,NaN,NaN,NaN,city =Dallas,NaN,NaN,android-app://com.google.android.gm/
12,Exp,2022-01-14 12:17:22,2022-01-14 12:31:14,0,73.237.188.225,100,13.866667,1,2022-01-14 12:31:15,R_1JE2On9wHg89sgJ,...,5,2,1,NaN,NaN,NaN,city =Stone Mountain,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_74...
13,Exp,2022-01-14 12:16:22,2022-01-14 12:50:41,0,184.2.81.253,100,34.300000,1,2022-01-14 12:50:42,R_2SlzPQY4LCZ3klS,...,1,1,1,NaN,NaN,NaN,city =Farmville,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_74...
14,Exp,2022-01-14 17:11:22,2022-01-14 17:29:19,0,73.209.75.165,100,17.950000,1,2022-01-14 17:29:20,R_OQqnG0muDQpfawN,...,2,2,1,No,No,NaN,city =Zion,NaN,NaN,android-app://com.google.android.gm/
15,Exp,2022-01-14 17:37:04,2022-01-14 17:43:37,0,174.245.114.94,100,6.533333,1,2022-01-14 17:43:38,R_3k6wc1yNcfFFsf7,...,1,2,1,NaN,NaN,NaN,city =Charlotte,NaN,NaN,android-app://com.google.android.gm/
16,Exp,2022-01-14 17:46:12,2022-01-14 18:21:41,0,98.114.236.198,100,35.466667,1,2022-01-14 18:21:42,R_3D11UOOJvSbdIGb,...,1,2,1,"Nothing, it was great!",Maybe add a discussion forum eventually,NaN,city =Willow Grove,NaN,NaN,android-app://com.google.android.gm/


####  Screening Survey
In addition to procedure above, we can also use the relevantID feature from qualtrics, we can find out the likeliness a survey is duplicated.
- https://www.qualtrics.com/support/survey-platform/survey-module/survey-checker/fraud-detection/

In [147]:
cDuplicate=screening['Q_RelevantIDDuplicateScore'] > 75 
cBot=screening['Q_RelevantIDFraudScore'] > 30

In [148]:
#create 'selectAll' col that checks if participant selects all options on a multi-choice question
expDf['selectAll']=expDf["E-Peri_Meno_Exper_p2"].apply(lambda x: (getLsLen(x)))
# check if the zipcode entered matches the state entered (experimental condition)
expDf['zipcodeState'] = expDf['E-Information_9_p2'].astype('int') #.apply(lambda x: findState(x))
expDf['E-Information_8_p2'] = expDf['E-Information_8_p2'].apply(lambda x: str(x)).apply(lambda x: x.upper())
expDf['zipcodeX_p2']=expDf['E-Information_8_p2'].isin(expDf.zipcodeState) 
#calculate percentage of NAs for each condition
expDf['percentNA_p2'] = expDf.iloc[:,27:].isna().sum(axis = 1)/len(expDf.iloc[:,27:].columns)
#check if street address has quotation marks around
expDf['QuoteMarks']= expDf['E-Information_5_p2'].apply(lambda x: containQuoteMarks(x))
#now we can begin to run the fraud detection procedure 
recaptcha = expDf['Q_RecaptchaScore_p2'] < 0.8 #recaptcha accuracy less than 80%
cond1=expDf["E-Peri_Meno_Exper_p2"].str.contains(',11') #select other options even though selected 'I refuse to answer'
cond2=expDf["E-ATT-CHECK1_p2"] != 1 #Did not select 'Strongly Agree'
cond3=expDf["E-ATT-CHECK2_p2"] != 5 #Did not select 'Strongly Disagree'
cond4=expDf['E-Web_Tools_p2'].str.contains(',14')#select other options even though selected 'I refuse to answer'

cond6 = ~expDf['Randomization_2_p2'].isin(list(np.arange(61, 81, 1)))  #age out of range
cond7 = expDf['E-gender_p2'] != 1 #not female
cond8=expDf.duplicated(['IPAddress_p2']) #IP address duplicated
cond9=expDf['durationMins_p2'] <=2 #finished survey in less than 2 mins
#cond10 = expDf['zipcodeX_p2'] != True #entered zipcode's state does not match entered state
cond11=expDf['RecipientEmail_p2'].str.isupper()
cond12 = expDf['selectAll'] == 11
expDf

,Cond_p2,StartDate_p2,EndDate_p2,Status_p2,IPAddress_p2,Progress_p2,durationMins_p2,Finished_p2,EndVisitTs_p2,ResponseId_p2,...,QCity_ExportTag_p2,cityName_p2,Email_p2,Q_BallotBoxStuffing_p2,Referer_p2,selectAll,zipcodeState,zipcodeX_p2,percentNA_p2,QuoteMarks
2,Exp,2022-01-14 00:01:58,2022-01-14 00:35:56,0,76.104.216.47,100,33.950000,1,2022-01-14 00:35:56,R_ReNNNAOqMc3gL6h,...,NaN,city =Bellingham,NaN,NaN,android-app://com.google.android.gm/,3,98257,False,0.097561,None
4,Exp,2022-01-14 03:09:44,2022-01-14 03:39:15,0,105.161.171.91,100,29.516667,1,2022-01-14 03:39:16,R_3NQg6A6Dd7X7toQ,...,NaN,city =New York,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_74...,3,92161,False,0.024390,None
5,Exp,2022-01-14 03:42:19,2022-01-14 04:24:43,0,86.106.87.110,100,42.383333,1,2022-01-14 04:24:43,R_3kHtyBUTqw6U7re,...,NaN,city =Miami,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_74...,3,92103,False,0.024390,None
10,Exp,2022-01-14 07:56:19,2022-01-14 11:15:09,0,76.183.86.133,100,198.833333,1,2022-01-14 11:15:10,R_26aobOw7DVENare,...,NaN,city =Dallas,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_74...,5,75023,False,0.250000,None
11,Exp,2022-01-14 08:45:12,2022-01-14 11:51:16,0,172.58.107.19,100,186.066667,1,2022-01-14 11:51:17,R_3GkW8Tq5b0KTlLB,...,NaN,city =Dallas,NaN,NaN,android-app://com.google.android.gm/,4,89048,False,0.152439,None
12,Exp,2022-01-14 12:17:22,2022-01-14 12:31:14,0,73.237.188.225,100,13.866667,1,2022-01-14 12:31:15,R_1JE2On9wHg89sgJ,...,NaN,city =Stone Mountain,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_74...,1,30058,False,0.140244,None
13,Exp,2022-01-14 12:16:22,2022-01-14 12:50:41,0,184.2.81.253,100,34.300000,1,2022-01-14 12:50:42,R_2SlzPQY4LCZ3klS,...,NaN,city =Farmville,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_74...,5,24530,False,0.048780,None
14,Exp,2022-01-14 17:11:22,2022-01-14 17:29:19,0,73.209.75.165,100,17.950000,1,2022-01-14 17:29:20,R_OQqnG0muDQpfawN,...,NaN,city =Zion,NaN,NaN,android-app://com.google.android.gm/,1,60087,False,0.097561,None
15,Exp,2022-01-14 17:37:04,2022-01-14 17:43:37,0,174.245.114.94,100,6.533333,1,2022-01-14 17:43:38,R_3k6wc1yNcfFFsf7,...,NaN,city =Charlotte,NaN,NaN,android-app://com.google.android.gm/,2,29356,False,0.146341,None
16,Exp,2022-01-14 17:46:12,2022-01-14 18:21:41,0,98.114.236.198,100,35.466667,1,2022-01-14 18:21:42,R_3D11UOOJvSbdIGb,...,NaN,city =Willow Grove,NaN,NaN,android-app://com.google.android.gm/,4,19090,False,0.115854,None


In [149]:
expFraud = []
expNoFraud = expDf
for x in [recaptcha , cond1, cond2 , cond3 , cond4 , cond6, cond7, cond8,cond9, cond11, cond12]:
    if (x.any()==True):
           expFraud.append({x.name:len(expDf[x])})
           #print(expDf[x][x.name])   

In [150]:
##### create 'selectAll' col that checks if participant selects all options on a multi-choice question
conDf['selectAll']=conDf["C-Peri_Meno_Experie_p2"].apply(lambda x: (getLsLen(x)))
# check if zipcode entered matches the state entered (control condition)
conDf['C-Information_9_p2']= (conDf['C-Information_9_p2'].str[:5])
conDf['zipcodeState'] = conDf['C-Information_9_p2'].astype('int')# .apply(lambda x: findState(x))
conDf['C-Information_8_p2'] = conDf['C-Information_8_p2'].apply(lambda x: str(x)).apply(lambda x: x.upper())
conDf['zipcodeX_p2']=conDf['C-Information_8_p2'].isin(conDf.zipcodeState)
#calculate percentage of NAs for each condition
conDf['percentNA_p2'] = conDf.iloc[:,27:].isna().sum(axis = 1)/len(conDf.iloc[:,27:].columns)
recaptcha = conDf['Q_RecaptchaScore_p2'] < 0.8 #recaptcha accuracy less than 80%

cond1=conDf["C-Peri_Meno_Experie_p2"].str.contains(',11') #select other options even though selected 'I refuse to answer'
cond2=conDf["C-ATT-CHECK1_p2"] != 1 #Did not select 'Strongly Agree'
cond3=conDf["C-ATT-CHECK2_p2"] != 5 #Did not select 'Strongly Disagree'
#cond5=conDf['C36.ATT CHECK_p2'].isna() #Did not select yes
cond6 = ~conDf['Randomization_2_p2'].isin(list(np.arange(67, 82, 1))) #age out of range
cond7 = conDf['C-Gender_p2'] != 1 #not female
cond8=conDf.duplicated(['IPAddress_p2']) #IP address duplicated
cond9=conDf['durationMins_p2'] <=2 #finished survey in less than 2 mins
#cond10 = conDf['zipcodeX_p2'] != True #entered zipcode's state does not match entered state
cond4=conDf['RecipientEmail_p2'].str.isupper() #all Caps, was the email variable removed?
cond11 = conDf['selectAll'] == 11

conFraud = []
conNoFraud = conDf
for x in [recaptcha, cond2 , cond3 , cond4 , cond6, cond7, cond8,cond9, cond11]:
    if (x.any()==True):
        conFraud.append({x.name: len(conNoFraud[x])})

#### How many fraudulent users today?

In [151]:
#percentFraudEachVar = pd.DataFrame.from_dict(conFraud + expFraud).sum()/len(qualtrics_data)
#print(pd.DataFrame.from_dict(conFraud + expFraud).sum())
#totalPercentFraud=pd.DataFrame.from_dict(totalConFraud + totalExpFraud).sum().sum()/len(qualtrics_data)
##print(totalPercentFraud,"\n", percentFraudEachVar)

#### How long does it take for participant to move from first o second survey?
We need to merge the first and second survey using `IPAddress` var to calculate the duration taken to complete the survey. Rename var `IPAddress_p2` to `IPAddress` so that they are mergeable. The merged df is called `p1p2df`. 

In [152]:
screening['EndDate']=pd.to_datetime(screening['EndDate'])#when did they finished survey 1
qualtrics_data['StartDate_p2']=pd.to_datetime(qualtrics_data['StartDate_p2'])#when did they start survey 2
qualtrics_data['EndDate_p2']=pd.to_datetime(qualtrics_data['EndDate_p2'])
#rename column from p2 survey to match columns on IP Address
qualtrics_data=qualtrics_data.rename(columns= {'IPAddress_p2':'IPAddress'})
p1p2df = pd.merge(qualtrics_data, screening, on='IPAddress')  

In [153]:
conCols = list(conDf.columns[30:-4])
expCols = list(expDf.columns[30:-10])
varE = []
for x in expCols:
    if x.startswith('E'):
        s = x.replace('E', 'I', 1)
        varE.append(s)
varC = []
for x in conCols:
    if x.startswith('C'):
        s = x.replace('C', 'I', 1)
        varC.append(s)


# rename variables so that they are the same to the control's wording 
newColNameE = list(expDf.columns[:30]) + varE+  list(expDf.columns[-10:])
expDf=expDf.iloc[:, :].set_axis([newColNameE], axis=1)
expDf=expDf.rename(columns= #rename cols
                                       {'I-gender_p2': 'I-Gender_p2',
                                        'I37.UTI_p2': 'I37.UI_p2',
                                       'I-Job_Status_p2': 'I-Job Status_p2',
                                       'I-Peri_Meno_Exper_p2':'I-Peri_Meno_Experie_p2',
                                       'I-Meno_Stage_Tran_p2': 'I-Meno_Stage_Trans_p2',
                                       'I-Menopause_Stage_p2':'I-Meno_Stage_p2'}) 

# rename variables so that they are the same 
newColNameC = list(conDf.columns[:30]) + varC+  list(conDf.columns[-9:])
conDf=conDf.iloc[:, :].set_axis([newColNameC], axis=1)

#### Which variables are in control condition but not in experimental condition?
`set(varC) - set(varE)
{'I-Email_Link_p2', #specific to control
  'I-Gender_p2',
 'I-Job Status_p2',
 'I-Meno_Stage_Trans_p2',
 'I-Meno_Stage_p2',
 'I-OtherWebsite1_p2', #specific to control
 'I-OtherWebsite2_p2', #specific to control
 'I-Peri_Meno_Experie_p2',
 'I-Website1_p2', #specific to control
 'I-Website2_p2', #specific to control
 'I37.UI_p2'}`

#### Which variables are in experimental  condition but not in  control condition?
`{'I-Job_Status_p2',
 'I-Meno_Stage_Tran_p2',
 'I-Menopause_Stage_p2',
 'I-Peri_Meno_Exper_p2',
 'I-Web_Tools_p2', #specific to exp
 'I-Website_Link_p2', #specific to exp
 'I-gender_p2',
 'I3.1-Useability_p2', #specific to exp
 'I37.UTI_p2'}`

In [154]:
allD = pd.concat([expDf, conDf])
allD=allD.drop(['zipcodeState', 'zipcodeX_p2'], axis=1)

C:\Users\huixin\anaconda3\lib\site-packages\pandas\core\generic.py:4147: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [155]:
# create csv for today's data
allD.to_csv(pilot_file_loc + "cleaned_data/cleaned" + today + "_p2.csv")